## Sheet 2
Zu Beginn werden Pakete in der virtuellen Umgebung installiert, die benötigt werden. Beispielsweise "pandas" über den Befehl "%pip install pandas". Anschließend werden benötigte Pakete importiert. Hier nur mal beispielsweise ein paar Pakete.

In [1]:
# Install pandas if not already installed
%pip install pandas
%pip install matplotlib
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

/usr/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.
/usr/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pandas'

Zunächst werden alle allgemeinen Konstanten und Funktionen definiert.

In [11]:
# Setting up some constants
# fine structure constant
alpha = 1/137
# Mass of the electron in MeV
m_e = 0.511
# Radius of the electron in 1/fm
r_e = 1.44/m_e
# Radius of the electron in cm
r_e = r_e * 1e-13
# Avogadro's number
N_A = 6.022*10**23

# Coulomb correction function for electron/positron
def f_coul(Z):
    a = Z/137
    f = a**2*((1+a**2)**(-1)+0.20206-0.0369*a**2+0.0083*a**4-0.002*a**6)
    return f

# number of atoms in 1 cm^3
def rho_to_N(rho, A):
    # Calculate number of atoms per cm^3
    N = rho * N_A / A
    return N
# Sollte 2,7*10^19 sein

# Function for the radiative correction
def phi_rad(Z, r_e, alpha, E0, m_e, shield_ind):
    # Calculate common factor
    cf = 4 * Z**2 * r_e**2 * alpha
    # Calculate the radiative correction
    if shield_ind == 0:
        phi = cf * (np.log(2 * E0 / m_e) - 1/3 - f_coul(Z))
    elif shield_ind == 1:
        phi = cf * (np.log(183*Z**(-1/3)) + 1/18 - f_coul(Z))
    return phi

# Function for the energy loss
def dE_dx(E0, Z, A, rho, r_e, alpha, m_e, shield_ind):
    # Calculate the energy loss
    dE = rho_to_N(rho, A) * E0 * phi_rad(Z, r_e, alpha, E0, m_e, shield_ind)
    return dE

# Approximation of the critical energy for electrons
def E_crit_approx(Z):
    # Calculate the critical energy using an approximation
    E_crit = 800/(Z+1.2)
    return E_crit

# Simplified radiation length function
def L_rad(Z, A):
    # Calculate the radiation length
    X0 = 716.4 * A / (Z*(Z+1) * np.log(287/np.sqrt(Z)))
    return X0

# Bethe-Heitler distribution function
def bh_distribution(E, E0, Z, A, material_length):
    # Calculate the length in terms of the radiation length
    t = material_length / L_rad(Z, A)
    # Calculate the relative energy
    x = E / E0
    # Calculate the distribution
    dN = -np.log(x)**(t/np.log(2)-1)/math.gamma(t/np.log(2))
    return dN 

Für die Aufgabenteile
1. vollständige Abschirmung, d.h. `shield_ind = 1`
2. keine Abschirmung, d.h. `shield_ind = 0`

In [10]:
# Initial energy in MeV
E0 = 1000
Z = 21 # Atomic number of Scandium
# Atomic mass of Scandium
A = 43
rho = 1.95*10**-3  # g/cm^3
# Avogadro's number
N_A = 6.022*10**23  # atoms/mol


# Exercise part a) complete shielding
print('Aufgabenteil a) ' "%10.3e"% dE_dx(E0, Z, A, rho, r_e, alpha, m_e, 1) + " MeV/cm")
# Exercise part b) no shielding
print('Aufgabenteil b) ' "%10.3e"% dE_dx(E0, Z, A, rho, r_e, alpha, m_e, 0) + " MeV/cm")

Aufgabenteil a)  1.179e-01 MeV/cm
Aufgabenteil b)  2.209e-01 MeV/cm


Aufgabenteil c): Die Voraussetzung der Formel muss geprüft werden, d.h. $E_0 >> 137 m_c \cdot c^2 \cdot Z^{1/3}$.

In [23]:
print('Aufgabenteil d) ' "%1.1f"% E_crit_approx(Z) + ' MeV')

Aufgabenteil d) 36.0 MeV


Die kritisch Teilchenenergie ist die Energie, bei der der Energieverlust durch Strahlung gleich dem Energieverlust durch Streuung entspricht. Elektronen bzw. Myonen mit E = 1 GeV liegen somit immer oberhalb der kritischen Energie.
Man müsste hier den WQ über die Abschätzung aus dem Hinweis nehmen und das Verhältnis von $E_c(e^-)/E_c(\mu)$ betrachtet. Daraus folgert man dann $E_c(\mu)\approx 1,5$ TeV.

In [30]:
Z = 14 # Atomic number of Silicon
A = 28
rho = 2.33*10**-3  # g/cm^3

# number of slices
n_slices = 6
# width of the slices
slice_width = 300 # in µm
# number of slices in cm
n_slices_cm = n_slices * slice_width * 1e-4
# Initial energy in MeV
E0 = 1000

# Calculate the probability of a 50% energy loss after 6 slices
print('Aufgabenteil e) ' "%f"% bh_distribution(E0/2, E0, Z, A, n_slices_cm) + ' %')
# Calculate the probability of a 50% energy loss after 12 slices
print('Aufgabenteil f) ' "%f"% bh_distribution(E0/2, E0, Z, A, 2*n_slices_cm) + ' %')

Aufgabenteil e) 3.748092 %
Aufgabenteil f) 3.056652 %


Aufgabe 2 betrachtet Energieauflösungen, um die $K_\alpha$-Linie $E_\gamma\approx 5.9$ keV mit einer Energieauflösung von $\Delta E_{\text{rel.}}<20\%$ zu messen.